In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Load data

from utils import downsample_volume, epfl_deconv_data

psf = []
downsampling_factor = 2

dx = np.array([0.0645, 0.0645, 0.16]) # um
dx = dx * downsampling_factor

gpu = False
for channel in range(3):
    _, psf_ = epfl_deconv_data(channel)
    psf_ = downsample_volume(psf_, downsampling_factor)

    if gpu:
        import cupy as xp
    else:
        xp = np

    psf_ = xp.asarray(psf_)

    # Same preprocessing as in Scico
    psf_ /= psf_.sum()
    psf.append(psf_)
psf = np.stack(psf)

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(3, 5))
for i in range(3):
    for j in range(3):
        axs[i, j].imshow(psf[i].sum(j), cmap="viridis_r")

fig.tight_layout()

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(10, 5))
for i in range(3):
    psf_ = psf[i].copy()
    psf_[psf_ < np.percentile(psf_.ravel(), 99)] = 0.
    for j in range(3):
        ids = [0, 1, 2]
        ids.remove(j)
        axs[i, j].plot(psf_.sum(tuple(ids)))

fig.tight_layout()

In [ ]:

psf_s = []
for i in range(3):
    psf_ = psf[i].copy()
    psf_[psf_ < np.percentile(psf_.ravel(), 99)] = 0.
    trims = np.zeros((3, 2), dtype=int)
    for j in range(3):
        ids = [0, 1, 2]
        ids.remove(j)
        psf_sum = psf_.sum(tuple(ids))
        trims[j][0] = len(psf_sum) - len(np.trim_zeros(psf_sum, trim='f'))
        trims[j][1] = len(psf_sum) - len(np.trim_zeros(psf_sum, trim='b'))

    psf_s.append(psf[i][trims[0,0]:-trims[0,1], trims[1,0]:-trims[1,1], trims[2,0]:-trims[2,1]])

fig, axs = plt.subplots(3, 3, figsize=(3, 5))
for i in range(3):
    for j in range(3):
        axs[i, j].imshow(psf_s[i].sum(j), cmap="viridis_r")
fig.tight_layout()